In [1]:
import pandas as pd

import plotly.express as px  # (version 4.7.0)
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import dash  # (version 1.12.0) pip install dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import functions.functions_data

from flask import Flask
import os

pd.set_option('display.max_columns', None) # To display all columns

In [2]:
dfs_country = functions.functions_data.get_data("data/CMU_Global_data/Full_Survey_Data/country/smooth/", "country")

Read in data completed.


In [3]:
countries = pd.concat(dfs_country, ignore_index=True)

In [4]:
countries = functions.functions_data.insert_month(countries)

Created month column.


In [5]:
countries_overall = countries[countries["gender"]=="overall"]
countries_overall = countries_overall[countries_overall["age_bucket"]=="overall"]

In [6]:
countries_16072020 = countries_overall[countries_overall["date"]=="2020-07-16"]

In [7]:
countries_16102020 = countries_overall[countries_overall["date"]=="2020-10-16"]

# Basic Map

In [11]:
fig = px.choropleth(
        data_frame = countries_16072020,
        locations="GID_0",
        color="rolling_total_responses",
        hover_name="country_agg",
        color_continuous_scale=px.colors.sequential.BuPu)
fig.show("browser")

# Creating dashboard

In [8]:
app = dash.Dash(__name__)

# ------------------------------------------------------------

df = countries[countries["age_bucket"]=="overall"]
df = df[df["date"]=="2020-07-16"]
df = df.groupby(["country_agg", "GID_0","gender", "date"])[["rolling_total_responses","smoothed_pct_cli_weighted"]].mean()

#--------------------------------------------------------------------------------
#App layout
app.layout = html.Div([
    
    html.H1("Dashboard for Capstone", style={"text-align": "centre"}),
    
    dcc.Dropdown(id="slct_gender",
                options=[
                    {"label": "Overall", "value": "overall"},
                    {"label": "Female", "value": "female"},
                    {"label": "Male", "value": "male"}],
                multi=False,
                value="overall",
                style={"width": "40%"}
                ),
    
    dcc.Dropdown(id="data",
                options=[
                    {"label": "Number of responses", "value": "rolling_total_responses"},
                    {"label": "Individuals with COVID-like illness", "value": "smoothed_pct_cli_weighted"}],
                multi=False,
                value="rolling_total_responses",
                style={"width": "40%"}
                ),
    
    html.Div(id="ouput_container", children=[]),
    html.Br(),
    
    dcc.Graph(id="my_covid_map", figure={})
    
])



#----------------------------------------------------------------------------
#Connect the plotly graphs with Dash Components

@app.callback(
    [Output(component_id = "output_container", component_property="children"),
        Output(component_id = "my_covid_map", component_property="figure")],
    [Input(component_id = "slct_gender", component_property="value"),
        Input(component_id = "data", component_property="value")]
)
def update_graph(option_slct1, option_slct2):
    print(option_slct1, option_slct2)
    print(type(option_slct1, option_slct2))
    
    container = "The user chose {} for the gender and to display {}".format(option_slct1, option_slct2)
    
    dff = df.copy
    dff = dff[dff["gender"]==option_slct1]
    dff = dff[option_slct2]
    
    #Plotly Express
    fig = px.choropleth(
        data_frame = dff,
        locations="GID_0",
        color=option_slct2,
        hover_name="country_agg",
        color_continuous_scale=px.colors.sequential.Plasma
    )
    
    return container, fig

#-------------------------------------------------------------------

if __name__ == "__main__":
    app.run_server(debug=True)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

/Users/lauradellantonio/opt/anaconda3/envs/capstone/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
